In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# 1. 시간정보를 파싱해서 연,월,일.. 등의 새로운 정보(칼럼) 추가
- 도움되지않는 월,일,주...등은 제거


- day와 month는 오히려 방해하는 요소  why?

> 이유1. day
train set의 날짜는 1-19일의 데이터가 있다, 각각의 카테고리로 인식
test set의 날짜는 20-31일의 데이터로 학습, 하지만 day가 겹치지않아 의미있게 쓰지 못했다.

> 이유2. month
위의 day처럼 train과 test의 값의 분포가 매우 다르다.
train : 월초(1-19)의 data
test : 월말(20-31)의 data
-> month를 추가할 시 안 좋은 학습이 더 과해진다.

> 종합 : tree model은 greedy형으로 학습한다 
1. 새로운 column이 들어올 시 분포를 보고 가치를 판단
2. (가치 높을 시) 다른 것들의 학습은 덜하고, 가중치(weight)를 day에게 양보해서 학습시킨다. (파이를 나눠먹는다)
3. 하지만 막상, train과 test는 day가 겹치지 않아 자원을 낭비함(과적합)

In [ ]:
pd.options.display.max_columns = 100

train = pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv", parse_dates = ["datetime"])
train["year"] = train["datetime"].dt.year
train["month"] = train["datetime"].dt.month
train["day"] = train["datetime"].dt.day
train["hour"] = train["datetime"].dt.hour
train["week"] = train["datetime"].dt.week
train["dayofweek"] = train["datetime"].dt.dayofweek
train["week_num"] = np.ceil(train["day"]/7)

train.head()

In [ ]:
# 상관계수보기

train.corr()["count"].sort_values(ascending = False)

# 2.정답값, 필요없는 칼럼 제거
- datetime은 위에서 새롭게 파싱했으므로 제거
- casual + registerd = count(정답값)도 전처리해주기

In [ ]:
print(train["year"].unique())  # 2011, 2012년
print(train["month"].unique()) # 1~12월
print(train["day"].unique())   # 0~19일
print(train["hour"].unique())  # 0~23시

In [ ]:
train2 = train.drop(["datetime","count","casual","registered","month","week","day"], axis=1)
train2.head()

In [ ]:
train

In [ ]:
# 통계량 추가하기

a = train.groupby("season")["count"].agg(["mean","count","max","std"])
a.rename(columns = {"mean": "s_mean", "count": "s_count","max":"s_max","std":"s_std"}, inplace=True)

b = train.groupby("holiday")["count"].agg(["mean","count","max","std"])
b.rename(columns = {"mean": "h_mean", "count": "h_count","max":"h_max","std":"h_std"}, inplace=True)

c = train.groupby("workingday")["count"].agg(["mean","count","max","std"])
c.rename(columns = {"mean": "w_mean", "count": "w_count","max":"w_max","std":"w_std"}, inplace=True)

d = train.groupby("weather")["count"].agg(["mean","count","max","std"])
d.rename(columns = {"mean": "wt_mean", "count": "wt_count","max":"wt_max","std":"wt_std"}, inplace=True)

e = train.groupby("hour")["count"].agg(["mean","count","max","std"])
e.rename(columns = {"mean": "hr_mean", "count": "hr_count","max":"hr_max","std":"hr_std"}, inplace=True)

f = train.groupby("dayofweek")["count"].agg(["mean","count","max","std"])
f.rename(columns = {"mean": "d_mean", "count": "d_count","max":"d_max","std":"d_std"}, inplace=True)

train2 = pd.merge(train2,a,on="season", how='left')
train2 = pd.merge(train2,b,on="holiday", how='left')
train2 = pd.merge(train2,c,on="workingday", how='left')
train2 = pd.merge(train2,d,on="weather", how='left')
train2 = pd.merge(train2,e,on="hour", how='left')
train2 = pd.merge(train2,f,on="dayofweek", how='left')
train2


#  3. test set도 train set과 같이 전처리

In [ ]:
test = pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv", parse_dates = ["datetime"])
test["year"] = test["datetime"].dt.year
#test["month"] = test["datetime"].dt.month
test["day"] = test["datetime"].dt.day
test["hour"] = test["datetime"].dt.hour
#test["week"] = test["datetime"].dt.week
test["week_num"] = np.ceil(test["day"]/7)
test["dayofweek"] = test["datetime"].dt.dayofweek

test.head()

In [ ]:
print(test["year"].unique())  # 2011, 2012년
# print(test["month"].unique()) # 1~12월
# print(test["day"].unique())   # 20~31일
print(test["hour"].unique())  # 0~23시

In [ ]:
test2 = test.drop(["datetime","day"], axis=1)
test2.head()

In [ ]:
aa = train.groupby("season")["count"].agg(["mean","count","max","std"])
aa.rename(columns = {"mean": "s_mean", "count": "s_count","max":"s_max","std":"s_std"}, inplace=True)

bb = train.groupby("holiday")["count"].agg(["mean","count","max","std"])
bb.rename(columns = {"mean": "h_mean", "count": "h_count","max":"h_max","std":"h_std"}, inplace=True)

cc = train.groupby("workingday")["count"].agg(["mean","count","max","std"])
cc.rename(columns = {"mean": "w_mean", "count": "w_count","max":"w_max","std":"w_std"}, inplace=True)

dd = train.groupby("weather")["count"].agg(["mean","count","max","std"])
dd.rename(columns = {"mean": "wt_mean", "count": "wt_count","max":"wt_max","std":"wt_std"}, inplace=True)

ee = train.groupby("hour")["count"].agg(["mean","count","max","std"])
ee.rename(columns = {"mean": "hr_mean", "count": "hr_count","max":"hr_max","std":"hr_std"}, inplace=True)

ff = train.groupby("dayofweek")["count"].agg(["mean","count","max","std"])
ff.rename(columns = {"mean": "d_mean", "count": "d_count","max":"d_max","std":"d_std"}, inplace=True)

test2 = pd.merge(test2,aa,on="season",how="left")
test2 = pd.merge(test2,bb,on="holiday",how="left")
test2 = pd.merge(test2,cc,on="workingday",how="left")
test2 = pd.merge(test2,dd,on="weather",how="left")
test2 = pd.merge(test2,ee,on="hour",how="left")
test2 = pd.merge(test2,ff,on="dayofweek",how="left")
test2


# 4.시각화
year,month,day 등 시간 별 count(정답값)의 그래프보기 ( categorical과 numerical 관계)

1. year-count 그래프 : 연도 별 count차이를 알 수 있고, outlier을 분리해줄 것이 필요
2. hour-count 그래프 : 시간대 별 count차이를 알 수 있고, 마찬가지로 outlier을 분리해줄 것이 필요
> - 아침7-8시와 저녁17~18, 22-04시에 많은 경우 : 평일의 출/퇴근, 야근시간 영향으로 유추
> - 아침~점심(10-16시)에 많은 outlier의 경우 : 주말의 경우로 유추

해주어야할 것 : 평일과 주말의 구분 필요 --> dayofweek(요일)칼럼 추가

In [ ]:
weekday_df = train[train["workingday"]==1] # 평일
print(weekday_df.shape)

weekend_df = train[train["workingday"]==0] # 주말
print(weekend_df.shape)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

a,b = plt.subplots(1,1,figsize=(20,12))
sns.boxplot(train["year"],train["count"])

a,b = plt.subplots(1,1,figsize=(20,12))
sns.boxplot(train["hour"],train["count"])

# 4-2 시각화2
1. 추가한 dayofweek(요일)-count 그래프 : outlier의 0-4와 5-6의 차이가 나타남 --> 평일,주말의 차이

2. 주말-count 그래프 : 위의 그래프의 outlier를 제대로 나타낸다.


In [ ]:
a,b = plt.subplots(1,1,figsize=(20,12))
sns.boxplot(train["dayofweek"],train["count"])

a,b = plt.subplots(1,1,figsize=(20,12))
sns.boxplot(weekend_df["hour"],weekend_df["count"])

In [ ]:
# 정답값 분포보기

a,b = plt.subplots(1,1,figsize=(20,12))
sns.distplot((train["count"]))

# count값의 분포그래프 : 값이 편향되어 있다
# --> 편향을 막기위해 log를 취해 편향을 해결할 필요가 있다.

In [ ]:
plt.figure(figsize=(20,12))
sns.distplot((np.log(train["count"])))

# 5. 머신러닝 모델 설계

bagging방식의 RandomForestRegressor로 데이터셋에 맞는 hyperparameter를 대략적으로 맞추고 baseline을 잡는다.

> - 통계량 feature를 추가하면서 columns이 많이 증가하고, 그로인해 bagging의 효율이 떨어져 baseline잡는데 어려움을 겪음

대신 robust한 catboost model로 baseline을 잡는다.


In [ ]:
# 결측치 처리
train2 = train2.fillna(-1)
test2 = test2.fillna(-1)

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor(n_estimators = 100, n_jobs = 4, random_state = 1)
# rf.fit(train2,np.log(train["count"]))
# result = rf.predict(test2)

from catboost import CatBoostRegressor
ca = CatBoostRegressor(iterations = 1000)
ca.fit(train2,np.log(train["count"]))
result1 = ca.predict(test2)


# 학습변수중요도 확인
- 각 칼럼의 학습된 정도를 확인가능 --> 중요도 확인가능

In [ ]:
pd.Series(ca.feature_importances_,index = train2.columns).sort_values(ascending = False)

# 6. 제출

In [ ]:
sub = pd.read_csv("/kaggle/input/bike-sharing-demand/sampleSubmission.csv")
sub.head()

In [ ]:
sub["count"] = np.exp(result)
sub.head()

In [ ]:
sub.to_csv("sub.csv", index=False)